In [ ]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import plot_tree
from sklearn.metrics import roc_curve, roc_auc_score
from tabulate import tabulate
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import trigrams
from  nltk import ngrams
import ipywidgets as widgets
from IPython.display import display

In [ ]:
#Loading the final cleaned datasets

merged_file_df = pd.read_TYPE('Merged_File')

In [ ]:
merged_file_df

In [ ]:
# DataFrame named 'merged_file_df' with a column 'Exam_Notes'
exam_notes = merged_file_df['Exam_Notes'].tolist()  # Convert 'Exam_Notes' column to a list

# Create a set to store unique words
vocabulary = set()

# Iterate over each note and split it into words
for note in exam_notes:
    words = note.lower().split()
    vocabulary.update(words)


# Convert the set back to a list for easier manipulation
vocabulary = list(vocabulary)


In [ ]:
vocabulary

In [ ]:
# import pandas as pd

# # DataFrame named 'merged_file_df' with columns 'Exam_Notes' and 'Disease_Severity'
# exam_notes = merged_file_df['Exam_Notes'].tolist()  # Convert 'Exam_Notes' column to a list
# disease_severity = merged_file_df['Disease_Severity'].tolist()  # Convert 'Disease_Severity' column to a list

# vocabulary = set()  # Create a set to store unique words

# # Step 1: Create vocabulary list
# for note in exam_notes:
#     words = note.lower().split()
#     vocabulary.update(words)

# # Step 2: Initialize dictionaries for word frequencies
# active_freq = {word: 0 for word in vocabulary}
# inactive_freq = {word: 0 for word in vocabulary}

# # Step 3-5: Calculate word frequencies for each class
# for note, severity in zip(exam_notes, disease_severity):
#     words = note.split()
#     word_freq = active_freq if severity == 'Active' else inactive_freq
#     for word in words:
#         if word in word_freq:
#             word_freq[word] += 1

# # # Step 6: Create new columns with word frequencies
# # merged_file_df['Active_Freq'] = merged_file_df['Exam_Notes'].apply(lambda x: sum(active_freq[word] for word in x.split()))
# # merged_file_df['Inactive_Freq'] = merged_file_df['Exam_Notes'].apply(lambda x: sum(inactive_freq[word] for word in x.split()))



In [ ]:


# Create a set of distinct lowercase words
word_set = set()
for note in merged_file_df['Exam_Notes']:
    words = note.lower().split()
    word_set.update(words)

# Create a new DataFrame with the distinct words
df_word = pd.DataFrame({'Word': list(word_set)})

# Display the resulting DataFrame
print(df_word)

            Word
0           5714
1          valve
2     throughout
3          large
4           5829
...          ...
1398        skin
1399        0034
1400  endoscopes
1401      degree
1402   travelers

[1403 rows x 1 columns]


In [ ]:
merged_file_df

# Stemming an lemmentization

In [ ]:
print(merged_file_df['Exam_Notes'].dtype)
merged_file_df['Image_Info'] = merged_file_df['Image_Info'].astype(str)

object


In [ ]:
# Convert Exam_Notes column to lowercase
merged_file_df['Exam_Notes_lower'] = [note.lower() for note in merged_file_df['Exam_Notes'].tolist()]
merged_file_df['Image_Info_lower'] = [note.lower() for note in merged_file_df['Image_Info'].tolist()]



In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')


lemmatizer = WordNetLemmatizer()

# Apply c on Exam_Notes column
merged_file_df['Exam_Notes_tokens'] = merged_file_df['Exam_Notes_lower'].apply(lambda x: [lemmatizer.lemmatize(token) for token in word_tokenize(x)])

# Apply lemmatization on Image_Info column
merged_file_df['Image_Info_tokens'] = merged_file_df['Image_Info_lower'].apply(lambda x: [lemmatizer.lemmatize(token) for token in word_tokenize(x)])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
merged_file_df

In [ ]:
# DataFrame named 'merged_file_df' with columns 'Exam_Notes' and 'Disease_Severity'
exam_notes = merged_file_df['Exam_Notes_tokens'].tolist()  # Convert 'Exam_Notes' column to a list
disease_severity = merged_file_df['Disease_Severity'].tolist()  # Convert 'Disease_Severity' column to a list

vocabulary = set()  # Create a set to store unique words

# Step 1: Create vocabulary list
for note in exam_notes:
    words = note.lower().split()
    vocabulary.update(words)

# Step 2: Initialize dictionaries for word frequencies
active_freq = {word: 0 for word in vocabulary}
inactive_freq = {word: 0 for word in vocabulary}

# Step 3-5: Calculate word frequencies for each class
for note, severity in zip(exam_notes, disease_severity):
    words = note.split()
    word_freq = active_freq if severity == 'Active' else inactive_freq
    for word in words:
        if word in word_freq:
            word_freq[word] += 1

In [ ]:
# import pandas as pd


# # Flatten the token lists in 'Exam_notes'
# flat_tokens = [token for tokens in merged_file_df['Exam_Notes_tokens'] for token in tokens]

# # Calculate the word frequencies based on 'Disease_severity'
# word_freq_active = pd.Series(flat_tokens).loc[merged_file_df['Disease_Severity'] == 'Active'].value_counts()
# word_freq_inactive = pd.Series(flat_tokens).loc[merged_file_df['Disease_Severity'] == 'Inactive'].value_counts()

# # Create new columns in the DataFrame
# new_df_1['Word_Freq_Active'] = merged_file_df['Exam_Notes_tokens'].apply(lambda x: sum(token in word_freq_active.index for token in x))
# new_df_1['Word_Freq_Inactive'] = merged_file_df['Exam_Notes_tokens'].apply(lambda x: sum(token in word_freq_inactive.index for token in x))


### On the basis of Column "Disease_Severity" which consists of two classes "Active" and "Inactive", make two columns "Active_Freq" and "Inactive_Freq" which contains word frequencies after matching from Vocabulary. Similar to feature extraction with frequencies in Sentiment analysis.

In [ ]:
# import pandas as pd

# # Create a list of tuples containing (word, active_freq, inactive_freq)
# word_freq_list = [(word, active_freq[word], inactive_freq[word]) for word in vocabulary]

# # Create a new DataFrame with columns 'Word', 'Active_Freq', 'Inactive_Freq'
# new_df = pd.DataFrame(word_freq_list, columns=['Word', 'Active_Freq', 'Inactive_Freq'])



import pandas as pd

# Create an empty list to store the word frequency data
word_freq_list = []

# Iterate over the vocabulary list
for word in vocabulary:
    # Check if the word exists in the active_freq and inactive_freq dictionaries
    if word in active_freq and word in inactive_freq:
        # Append the word and its corresponding frequencies to the list
        word_freq_list.append((word, active_freq[word], inactive_freq[word]))

# Create a new DataFrame with columns 'Word', 'Active_Freq', 'Inactive_Freq'
new_df = pd.DataFrame(word_freq_list, columns=['Word', 'Active_Freq', 'Inactive_Freq'])


In [ ]:
# # Flatten the token lists in 'Exam_notes'
# flat_tokens = [token for tokens in merged_file_df['Exam_Notes_tokens'] for token in tokens]

# # Calculate the token frequency
# token_freq = pd.Series(flat_tokens).value_counts()

# # Create a new DataFrame with columns 'Token' and 'Frequency'
# new_df_1 = pd.DataFrame({'Token': token_freq.index, 'Frequency': token_freq.values})


# Flatten the token lists in 'Exam_notes'
flat_tokens = [token for tokens in merged_file_df['Exam_Notes_tokens'] for token in tokens]

# Calculate the token frequency
token_freq = pd.Series(flat_tokens).value_counts()

# Create a new DataFrame with columns 'Token' and 'Frequency'
new_df_1 = pd.DataFrame({'Token': token_freq.index, 'Frequency': token_freq.values})

# Calculate the word frequencies based on 'Disease_Severity'
word_freq_active = merged_file_df.loc[merged_file_df['Disease_Severity'] == 'Active', 'Exam_Notes_tokens'].explode().value_counts()
word_freq_inactive = merged_file_df.loc[merged_file_df['Disease_Severity'] == 'Inactive', 'Exam_Notes_tokens'].explode().value_counts()

# Add new columns to the DataFrame
new_df_1['Word_Freq_Active'] = new_df_1['Token'].map(word_freq_active).fillna(0)
new_df_1['Word_Freq_Inactive'] = new_df_1['Token'].map(word_freq_inactive).fillna(0)


In [ ]:
new_df_1['Word_Freq_Active'] = new_df_1['Word_Freq_Active'].astype(int)
new_df_1['Word_Freq_Inactive'] = new_df_1['Word_Freq_Inactive'].astype(int)

In [ ]:
new_df_1

In [ ]:
# Sort the DataFrame by the 'Frequency' column in descending order
new_df_1 = new_df_1.sort_values(by='Frequency', ascending=True)

# Add the 'Rank' column
new_df_1['Rank'] = range(1, len(new_df_1) + 1)

# Rearrange the columns
new_df_1 = new_df_1[['Token', 'Frequency', 'Rank', 'Word_Freq_Active', 'Word_Freq_Inactive']]

In [ ]:
new_df_1

In [ ]:
Active_df = merged_file_df[merged_file_df['Disease_Severity'] == 'Active'].copy()

Inactive_df = merged_file_df[merged_file_df['Disease_Severity'] == 'Inactive'].copy()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Extract the text from the respective dataframes
active_texts = Active_df['Exam_Notes'].tolist()
inactive_texts = Inactive_df['Exam_Notes'].tolist()

# Concatenate the active and inactive texts
all_texts = Active_df['Exam_Notes'].tolist() + Inactive_df['Exam_Notes'].tolist()

# Create an instance of the TF-IDF vectorizer
vectorizer = CountVectorizer(ngram_range=(1, 1))

# Fit and transform the vectorizer on all texts
vectors = vectorizer.fit_transform(all_texts)

# Split the vectors into active and inactive vectors
active_vectors = vectors[:len(Active_df)]
inactive_vectors = vectors[len(Active_df):]

# Calculate the average vectors for each group
active_avg_vector = np.mean(active_vectors, axis=0)
inactive_avg_vector = np.mean(inactive_vectors, axis=0)

# Calculate the difference between the average vectors
difference = active_avg_vector - inactive_avg_vector

# Get the vocabulary (dictionary of feature indices to feature names)
vocabulary = vectorizer.vocabulary_

# Get the feature names from the vectorizer
feature_names = [word for word, index in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1])]

# Calculate the total frequency, active frequency, and inactive frequency of each word
total_freq = np.sum(vectors.toarray(), axis=0)
active_freq = np.sum(active_vectors.toarray(), axis=0)
inactive_freq = np.sum(inactive_vectors.toarray(), axis=0)

# Create a dataframe with word, frequency, active frequency, inactive frequency, and rank
new_df = pd.DataFrame({'Word': feature_names, 'Freq': total_freq, 'Active_Freq': active_freq, 'Inactive_Freq': inactive_freq, 'Rank': np.arange(1, len(feature_names) + 1)})

In [ ]:
new_df

In [ ]:
new_df.fillna(0)

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# List of clinical notes
clinical_notes = merged_file_df['Exam_Notes'].tolist()

# Tokenize the clinical notes
tokenized_notes = [word_tokenize(note.lower()) for note in clinical_notes]

# Train the Word2Vec model
model = Word2Vec(tokenized_notes, min_count=1)

Similar words to 'colon':
perianalrectal
0649
8446
findings
5742


In [ ]:
# Example usage
word = "active"
similar_words = model.wv.most_similar(word, topn=5)

print(f"Similar words to '{word}':")
for similar_word, similarity in similar_words:
    print(similar_word)

Similar words to 'active':
mayo
endo
uc
graded
simple


In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np

# List of clinical notes
clinical_notes = merged_file_df['Exam_Notes'].tolist()

# Tokenize the clinical notes
tokenized_notes = [word_tokenize(note.lower()) for note in clinical_notes]

# Train the Word2Vec model
model = Word2Vec(tokenized_notes, min_count=1)

# Calculate word rankings based on occurrence and similarity
word_occurrence = {}
word_rankings = {}

for note in tokenized_notes:
    for word in note:
        if word not in word_occurrence:
            word_occurrence[word] = 1
        else:
            word_occurrence[word] += 1
        if word not in word_rankings:
            word_embeddings = [model.wv[word] for word in note if word in model.wv]
            word_rankings[word] = np.mean(word_embeddings) * (1 / word_occurrence[word])

# Sort the words based on their rankings
sorted_words = sorted(word_rankings, key=word_rankings.get, reverse=True)

# Print the ranked words
for word in sorted_words:
    print(f"Word: {word}, Rank: {word_rankings[word]*100}")


In [ ]:
import plotly.express as px

# Create a DataFrame with the word ranks and disease severity
data = pd.DataFrame({'Word Rank': word_rankings, 'Disease Severity': disease_severity})

# Create a scatter plot using Plotly
fig = px.scatter(data, x='Word Rank', y='Disease Severity')

# Set the plot labels
fig.update_layout(xaxis_title='Word Rank', yaxis_title='Disease Severity')

# Show the plot
fig.show()


from the above code, make a dataframe with columns(word, as_active, as_inactive, ranks):
word: words from the above word
as_active: ranks of these words when found in Active_df(contains all the clinical notes for active patients)
as_inactive: ranks of these words when found in Inactive_df(contains all the clinical notes for inactive patients)
ranks: ranks from the above code

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np

# List of clinical notes
clinical_notes_active = Active_df['Exam_Notes'].tolist()
clinical_notes_inactive = Inactive_df['Exam_Notes'].tolist()

# Tokenize the clinical notes
tokenized_notes_active = [word_tokenize(note.lower()) for note in clinical_notes_active]
tokenized_notes_inactive = [word_tokenize(note.lower()) for note in clinical_notes_inactive]

# Train the Word2Vec model
model = Word2Vec(tokenized_notes_active + tokenized_notes_inactive, min_count=1)

# Calculate word rankings based on occurrence and similarity
word_occurrence = {}
word_rankings = {}

for note in tokenized_notes_active:
    for word in note:
        if word not in word_occurrence:
            word_occurrence[word] = 1
        else:
            word_occurrence[word] += 1
        if word not in word_rankings:
            word_embeddings = [model.wv[word] for word in note if word in model.wv]
            word_rankings[word] = (np.mean(word_embeddings) * (1 / word_occurrence[word]))*100

# Sort the words based on their rankings
sorted_words = sorted(word_rankings, key=word_rankings.get, reverse=True)

# Create the DataFrame with columns 'word' and active__ranks
data = {
    'word': sorted_words,
    'active__ranks': [word_rankings[word] for word in sorted_words]
}

ranks_active_df = pd.DataFrame(data)

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np

# List of clinical notes
clinical_notes_active = Active_df['Exam_Notes'].tolist()
clinical_notes_inactive = Inactive_df['Exam_Notes'].tolist()

# Tokenize the clinical notes
tokenized_notes_active = [word_tokenize(note.lower()) for note in clinical_notes_active]
tokenized_notes_inactive = [word_tokenize(note.lower()) for note in clinical_notes_inactive]

# Train the Word2Vec model
model = Word2Vec(tokenized_notes_active + tokenized_notes_inactive, min_count=1)

# Calculate word rankings based on occurrence and similarity
word_occurrence = {}
word_rankings = {}

for note in tokenized_notes_inactive:
    for word in note:
        if word not in word_occurrence:
            word_occurrence[word] = 1
        else:
            word_occurrence[word] += 1
        if word not in word_rankings:
            word_embeddings = [model.wv[word] for word in note if word in model.wv]
            word_rankings[word] = (np.mean(word_embeddings) * (1 / word_occurrence[word]))*100

# Sort the words based on their rankings
sorted_words = sorted(word_rankings, key=word_rankings.get, reverse=True)

# Create the DataFrame with columns 'word' and inactive__ranks
data = {
    'word': sorted_words,
    'inactive__ranks': [word_rankings[word] for word in sorted_words]
}

ranks_inactive_df = pd.DataFrame(data)

In [ ]:
ranks_active_df

In [ ]:
ranks_inactive_df

In [ ]:
merged_df = ranks_active_df.merge(ranks_inactive_df, on='word', how='left')


In [ ]:
merged_df = merged_df.fillna(0)


In [ ]:
merged_df.to_csv("Ranks_merged_df.csv")

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np

# List of inactive patient's clinical notes
inactive_notes = Inactive_df['Exam_Notes'].tolist()

# Tokenize the inactive patient's clinical notes
tokenized_inactive_notes = [word_tokenize(note.lower()) for note in inactive_notes]

# Calculate word rankings based on occurrence and similarity for inactive patient's clinical notes
word_occurrence_inactive = {}
word_rankings_inactive = {}

for note in tokenized_inactive_notes:
    for word in note:
        if word not in word_occurrence_inactive:
            word_occurrence_inactive[word] = 1
        else:
            word_occurrence_inactive[word] += 1
        if word not in word_rankings_inactive:
            word_embeddings_inactive = [model.wv[word] for word in note if word in model.wv]
            word_rankings_inactive[word] = np.mean(word_embeddings_inactive) * (1 / word_occurrence_inactive[word])

# Sort the words based on their rankings for inactive patient's clinical notes
sorted_words_inactive = sorted(word_rankings_inactive, key=word_rankings_inactive.get, reverse=True)

# Create a DataFrame with word, rank in active patient's clinical notes, rank in inactive patient's clinical notes
df_inactive_ranks = pd.DataFrame({'Word': sorted_words_inactive, 'Inactive_Rank': [word_rankings_inactive[word]*100 for word in sorted_words_inactive]})
df_inactive_ranks

In [ ]:
from collections import Counter

# Combine active and inactive clinical notes into a single list
all_notes = clinical_notes_active + clinical_notes_inactive

# Tokenize all clinical notes
tokenized_all_notes = [word_tokenize(note.lower()) for note in all_notes]

# Count the occurrences of each word
word_counts = Counter(word for note in tokenized_all_notes for word in note)

# Create a DataFrame with word and frequency columns
df_word_freq = pd.DataFrame({'word': word_counts.keys(), 'Frequency': word_counts.values()})

# Sort the DataFrame by frequency in descending order
df_word_freq = df_word_freq.sort_values('Frequency', ascending=False)


df_word_freq

In [ ]:
merged_df_freq_ranks = merged_df.merge(df_word_freq, on='word', how='left')
max(merged_df_freq_ranks['Frequency'])

19918

In [ ]:
merged_df_freq_ranks

In [ ]:
merged_df_freq_ranks.to_csv("Ranks_merged_Freq_df.csv")

In [ ]:
import plotly.express as px

# fig = px.scatter(merged_df, x='active__ranks', y='inactive__ranks', title='Word Rankings')
# fig.update_traces(marker=dict(color='blue'), mode='markers')
# fig.show()


fig = px.scatter(merged_df_freq_ranks, x='active__ranks', y='inactive__ranks', title='Word Rankings', text='word', color='inactive__ranks')
fig.update_traces(mode='markers', textposition='top center')
fig.update_layout(xaxis_title='Active Ranks', yaxis_title='Inactive Ranks')
fig.show()
# import plotly.io as pio

# Save the scatter plot as an HTML file
#pio.write_html(fig, 'scatter_plot.html')

In [ ]:
import plotly.express as px

# Generate Plotly scatter plot with size based on 'Freq' values
fig = px.scatter(merged_df_freq_ranks, x='active__ranks', y='inactive__ranks', size='Frequency', title='Word Rankings')

fig.update_traces(marker=dict(color='blue'), mode='markers')

fig.show()


### To create a new DataFrame consisting of the vocabulary and the corresponding word frequencies for "Active" and "Inactive" classes, you can use the code provided below. To add colors to distinguish between 'Active_Freq' and 'Inactive_Freq' in the bar chart, you can specify different colors for each bar.

In [ ]:
import plotly.express as px

# Create a copy of the DataFrame and convert the 'Word' column to string type
plot_df = new_df.copy()
plot_df['Word'] = plot_df['Word'].astype(str)

# Create the bar chart using Plotly
fig = px.bar(plot_df, x='Word', y=['Active_Freq', 'Inactive_Freq'],
             color_discrete_sequence=['blue', 'red'],
             labels={'value': 'Frequency', 'variable': 'Disease Severity'},
             title='Word Frequency by Disease Severity')

# Rotate x-axis labels for better visibility
fig.update_layout(xaxis_tickangle=-45)

# Show the chart
fig.show()


### Code that includes plots using Plotly to compare the frequencies of words with Higher frequency in the 'Active_Freq' column compared to the 'Inactive_Freq' column:

In [ ]:
# Filter words with lower frequency in Active_Freq than Inactive_Freq
lower_freq_words = new_df[new_df['Inactive_Freq'] < new_df['Active_Freq']]

# Plotting bar chart
fig = px.bar(lower_freq_words, x='Word', y=['Active_Freq', 'Inactive_Freq'],
             title='Comparison of Word Frequencies (Active vs. Inactive)',
             labels={'value': 'Frequency', 'variable': 'Disease Severity'},
             barmode='group')

fig.update_layout(xaxis_tickangle=-45)

# Show the plot
fig.show()


In [ ]:
import medspacy
import spacy

# Load the pre-trained MedSpacy model
nlp = medspacy.load()

# Initialize an empty list to store medical terms with lower frequency
medical_lower_freq_words = []

# Iterate over the words with lower frequency
for word in lower_freq_words:
    doc = nlp(word)  # Process the word with MedSpacy
    if any(ent._.is_medical for ent in doc.ents):
        medical_lower_freq_words.append(word)




In [ ]:
# Print the medical terms with lower frequency
for word in medical_lower_freq_words:
    print(word)

In [ ]:
# Load the pre-trained MedSpacy model
nlp = medspacy.load()

# Create a list to store medical words
medical_words = []

# Iterate over the words in the vocabulary
for word in vocabulary:
    doc = nlp(word)  # Process the word with MedSpacy
    if any(ent._.is_medical for ent in doc.ents):
        medical_words.append(word)

# Filter the DataFrame to include only medical words
medical_df = new_df[new_df['Word'].isin(medical_words)]

# Print the filtered DataFrame
print(medical_df)


Empty DataFrame
Columns: [Word, Active_Freq, Inactive_Freq]
Index: []


 to add medical terms to the MedSpacy model and train it on custom medical data, you can follow these steps:

1. **Collect Medical Corpus**: Gather a corpus of medical texts that cover the specific domain or medical terms you want to add. This could include medical journals, clinical notes, research papers, or any other relevant sources.

2. **Annotate Data**: Annotate the collected medical corpus with the entities you want to recognize, such as medical terms or specific medical concepts. You can use annotation tools like Prodigy, Brat, or even manually annotate the data.

3. **Convert Annotations to MedSpaCy Format**: Convert the annotated data into MedSpaCy format, which typically involves converting the annotations to the Doc and Span objects of MedSpaCy.

4. **Train MedSpaCy**: Use the converted annotated data to train a MedSpaCy model. This involves initializing a blank MedSpaCy model, loading the converted data, and running multiple training iterations to update the model's entity recognition capabilities.

5. **Evaluate and Fine-tune**: After training, evaluate the performance of the trained model on a held-out dataset. If necessary, fine-tune the model by iterating over the training and evaluation process until satisfactory performance is achieved.

6. **Test and Use**: Once the model is trained and performs well, you can use it to recognize medical terms in new text data.

In [ ]:
import spacy
import scispacy

# Load the scispacy pipeline
nlp = spacy.load("en_core_sci_md")

# Define the text to analyze
text = "ENCRYPTED"

# Process the text with the scispacy pipeline
doc = nlp(text)

# Extract medical terms from the document
medical_terms = [ent.text for ent in doc.ents if ent.label_ in ['CHEMICAL', 'DISEASE', 'SYMPTOM']]

# Print the extracted medical terms
for term in medical_terms:
    print(term)
